In [1]:
# parse output.txt

import re
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

file_path = "/home/pan251/accel-sim-framework-pim/output.txt"
done_cycles = {}
start_cycles = {}

color_array = px.colors.qualitative.Plotly

freq = 0
last_done = 0

xbar_to_layer = {}


with open(file_path, "r") as file:
    for line in file:
        match = re.match(r"core (\d*) xbar (\d*) done layer, (\d*)", line)
        if match:
            core = int(match.group(1))
            xbar_id = int(match.group(2))
            cycle = int(match.group(3))
            xbar_id = xbar_id + core * 256
            if xbar_id not in done_cycles:
                done_cycles[xbar_id] = []
            done_cycles[xbar_id].append(cycle)
            last_done = cycle

        match = re.match(r"core (\d*) xbar (\d*) start compute, (\d*)", line)
        if match:
            core = int(match.group(1))
            xbar_id = int(match.group(2))
            cycle = int(match.group(3))
            xbar_id = xbar_id + core * 256
            if xbar_id not in start_cycles:
                start_cycles[xbar_id] = []
            start_cycles[xbar_id].append(cycle)

        match = re.match(r"-gpgpu_clock_domains (\d*.\d*):.*", line)
        if match:
            freq = float(match.group(1)) * 1e6

        match = re.match(r"layer (.*) mapped to core (\d*) xbar (\d*)", line)
        if match:
            layer = match.group(1)
            core = int(match.group(2))
            xbar_id = int(match.group(3))
            xbar_id = xbar_id + core * 256
            xbar_to_layer[xbar_id] = "layer " + layer


In [ ]:
start_cycles

In [2]:
# plot

to_plot = pd.DataFrame(columns=["layer", "xbar", "start", "done", "duration", "color", "text"])
for xbar, starts in start_cycles.items():
    if xbar not in done_cycles:
        continue
    if (len(starts) != len(done_cycles[xbar])):
        done_cycles[xbar].append(last_done)
    assert(len(starts) == len(done_cycles[xbar]))
    dones = done_cycles[xbar]
    for index, start in enumerate(starts):
        start = start
        done = dones[index]
        start = start / freq * 1e3
        done = done / freq * 1e3
        duration = (done - start)
        # format
        # duration = "{:.5f}".format(duration)
        
        to_plot = pd.concat([to_plot, pd.DataFrame({
            "layer": [xbar_to_layer[xbar]],
            "xbar": [str(xbar)], 
            "start": [start], 
            "duration": [duration], 
            "done": [done], 
            "color": [color_array[index % len(color_array)]],
            "text": [f"{duration:.2f} ms"]
            })])
        
fig = go.Figure()
fig.add_trace(go.Bar(
    x = to_plot["duration"],
    y = [to_plot["layer"],"xbar " + to_plot["xbar"]],
    orientation = "h",
    base = to_plot["start"],
    marker_color = to_plot["color"],
    # add text duration
    text = to_plot["text"],
)
)

# change text font size

# change size
# fig.update_layout(
#     autosize=False,
#     width=2000,
#     height=len(xbar_to_layer) * 30,
#     margin=dict(l=0, r=0, t=0, b=0)
# )

# change x range
fig.update_xaxes(range=[0, to_plot["start"].max() * 1.1])

import plotly.io as pio
pio.renderers.default = 'browser'
# write jpg
# fig.write_image("pip.jpg", width=1920, height=1080, scale=2)
fig.write_html("./pip.html")
fig.show()

In [1]:
import onnx

model = onnx.load("/home/pan251/accel-sim-framework-pim/resnet50.onnx")

with open("/home/pan251/accel-sim-framework-pim/serialized.model", "wb") as f:
  f.write(model.graph.SerializeToString())


In [12]:
to_plot["duration"]

0    14.674277
0    14.674275
0    14.674275
0    14.674275
0    14.674275
       ...    
0    86.071648
0    54.374099
0    78.589089
0    54.427327
0    20.335157
Name: duration, Length: 964, dtype: object

In [13]:
import torch
from transformers import BertModel, BertTokenizer

# List to store operations
operations = []

# Save original functions
original_matmul = torch.matmul
original_linear = torch.nn.functional.linear

# Monkey-patch torch.matmul
def matmul_hook(input, other):
    output = original_matmul(input, other)
    operations.append({
        'op_type': 'MatMul',
        'input_ids': (id(input), id(other)),
        'output_id': id(output),
        'input_shapes': (input.shape, other.shape),
        'output_shape': output.shape,
        'layer_name': current_layer_name  # Add current layer name
    })
    return output

torch.matmul = matmul_hook

# Monkey-patch torch.nn.functional.linear
def linear_hook(input, weight, bias=None):
    output = original_linear(input, weight, bias)
    operations.append({
        'op_type': 'Linear',
        'input_ids': (id(input), id(weight), id(bias) if bias is not None else None),
        'output_id': id(output),
        'input_shapes': (input.shape, weight.shape, bias.shape if bias is not None else None),
        'output_shape': output.shape,
        'layer_name': current_layer_name  # Add current layer name
    })
    return output

torch.nn.functional.linear = linear_hook

# Function to register forward hooks
current_layer_name = None

def forward_hook(layer_name):
    def hook(module, input, output):
        global current_layer_name
        current_layer_name = layer_name  # Update the global variable to track the current layer name
    return hook

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Register hooks on all layers
for name, module in model.named_modules():
    module.register_forward_hook(forward_hook(name))

# Prepare sample input
text = "Hello, this is a sample input to test the BERT model."
inputs = tokenizer(text, return_tensors='pt')

# Run the model
outputs = model(**inputs)

# Restore original functions
torch.matmul = original_matmul
torch.nn.functional.linear = original_linear

# Build a mapping from output tensor IDs to operations
tensor_id_to_op = {}
for op in operations:
    tensor_id_to_op[op['output_id']] = op

# Assign operation IDs
for idx, op in enumerate(operations):
    op['op_id'] = idx

# Build dependencies
for op in operations:
    op['depends_on'] = []
    for input_id in op['input_ids']:
        if input_id in tensor_id_to_op:
            input_op = tensor_id_to_op[input_id]
            op['depends_on'].append(input_op['op_id'])

# Print the dependency graph
for op in operations:
    print(f"Operation {op['op_id']} ({op['op_type']}) on layer '{op['layer_name']}'")
    print(f"  Input shapes: {op['input_shapes']}")
    print(f"  Output shape: {op['output_shape']}")


Operation 0 (Linear) on layer 'embeddings'
  Input shapes: (torch.Size([1, 15, 768]), torch.Size([768, 768]), torch.Size([768]))
  Output shape: torch.Size([1, 15, 768])
Operation 1 (Linear) on layer 'encoder.layer.0.attention.self.query'
  Input shapes: (torch.Size([1, 15, 768]), torch.Size([768, 768]), torch.Size([768]))
  Output shape: torch.Size([1, 15, 768])
Operation 2 (Linear) on layer 'encoder.layer.0.attention.self.key'
  Input shapes: (torch.Size([1, 15, 768]), torch.Size([768, 768]), torch.Size([768]))
  Output shape: torch.Size([1, 15, 768])
Operation 3 (MatMul) on layer 'encoder.layer.0.attention.self.value'
  Input shapes: (torch.Size([1, 12, 15, 64]), torch.Size([1, 12, 64, 15]))
  Output shape: torch.Size([1, 12, 15, 15])
Operation 4 (MatMul) on layer 'encoder.layer.0.attention.self.dropout'
  Input shapes: (torch.Size([1, 12, 15, 15]), torch.Size([1, 12, 15, 64]))
  Output shape: torch.Size([1, 12, 15, 64])
Operation 5 (Linear) on layer 'encoder.layer.0.attention.self'

/home/pan251/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [1]:
import torch
from torchviz import make_dot
from transformers import BertModel, BertTokenizer

# Initialize BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Sample input text
text = "This is an example input for BERT."
inputs = tokenizer(text, return_tensors='pt')

# Perform forward pass
outputs = model(**inputs)

# Visualize the computation graph
# We take the last hidden state output for visualization
graph = make_dot(outputs.last_hidden_state, params=dict(model.named_parameters()))

# Render the graph and save it as a PNG image
graph


/home/pan251/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


'bert_computation_graph.png'